In [ ]:
# Introducción
#### 0.1.1 - 2025 - 01 - 01
#### Dr. Marco Aceves 
#### rev en Jupyter Notebook
#### Código como ejemplo como parte del libro:
#### de 0 a 100 en Inteligencia Artificial
#### 4_ID3

#¿Cómo funciona el algoritmo ID3?
El algoritmo ID3 se basa en la selección recursiva de atributos que dividen el conjunto de datos en la mejor manera posible para clasificar la clase de los ejemplos. La corte de un atributo se realiza utilizando la entropía y el ganancia de información.

##Pasos del algoritmo ID3:
###Elegir el atributo más relevante:

Para decidir qué atributo dividir, ID3 usa la Ganancia de Información que evalúa cuánto reduce la entropía de un conjunto de datos al dividirlo por un atributo específico.

La Entropía mide la incertidumbre o la impureza de un conjunto de datos. Si un conjunto tiene solo ejemplos de una clase, su entropía será 0. Si tiene una mezcla igual de clases, la entropía será alta.

###Dividir el conjunto de datos:

Después de elegir el atributo que ofrece la mayor ganancia de información, el conjunto de datos se divide en subgrupos con base en los posibles valores de ese atributo.

###Repetir recursivamente:

El proceso se repite para cada subconjunto resultante, seleccionando en cada paso el atributo con la mayor ganancia de información hasta que:
Todos los elementos de un subconjunto tengan la misma clase.
No haya más atributos para dividir.

###Crear la hoja de decisión:

Cuando el proceso de división no puede continuar, se asigna una clase a la hoja del árbol, que representa la decisión final para esos datos.


In [7]:
import pandas as pd

#Cargar el dataset Titanic
data = pd.read_csv('titanic.csv')

#Mostrar las primeras filas del conjunto de datos
print("Primeras filas del dataset:")
print(data.head())

#Describir las columnas y tipos de datos
print("\nDescripción de las columnas:")
print(data.info())

Primeras filas del dataset:
   pclass  survived                                             name     sex  \
0       1         1                    Allen, Miss. Elisabeth Walton  female   
1       1         1                   Allison, Master. Hudson Trevor    male   
2       1         0                     Allison, Miss. Helen Loraine  female   
3       1         0             Allison, Mr. Hudson Joshua Creighton    male   
4       1         0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female   

       age  sibsp  parch  ticket      fare    cabin embarked boat   body  \
0  29.0000      0      0   24160  211.3375       B5        S    2    NaN   
1   0.9167      1      2  113781  151.5500  C22 C26        S   11    NaN   
2   2.0000      1      2  113781  151.5500  C22 C26        S  NaN    NaN   
3  30.0000      1      2  113781  151.5500  C22 C26        S  NaN  135.0   
4  25.0000      1      2  113781  151.5500  C22 C26        S  NaN    NaN   

                         home.dest

In [8]:
#Eliminar columnas irrelevantes
data = data.drop(columns=['name', 'ticket', 'cabin'])

#Rellenar valores faltantes en la columna 'Age' con la media
data['age'] = data['age'].fillna(data['age'].mean())

#Rellenar valores faltantes en la columna 'Embarked' con el valor más frecuente
data['embarked'] = data['embarked'].fillna(data['embarked'].mode()[0])

#Convertir variables categóricas a numéricas
data['sex'] = data['sex'].map({'male': 0, 'female': 1})
data['embarked'] = data['embarked'].map({'C': 0, 'Q': 1, 'S': 2})

#Verificar el resultado del preprocesamiento
print("\nDatos después del preprocesamiento:")
print(data.head())


Datos después del preprocesamiento:
   pclass  survived  sex      age  sibsp  parch      fare  embarked boat  \
0       1         1    1  29.0000      0      0  211.3375         2    2   
1       1         1    0   0.9167      1      2  151.5500         2   11   
2       1         0    1   2.0000      1      2  151.5500         2  NaN   
3       1         0    0  30.0000      1      2  151.5500         2  NaN   
4       1         0    1  25.0000      1      2  151.5500         2  NaN   

    body                        home.dest  
0    NaN                     St Louis, MO  
1    NaN  Montreal, PQ / Chesterville, ON  
2    NaN  Montreal, PQ / Chesterville, ON  
3  135.0  Montreal, PQ / Chesterville, ON  
4    NaN  Montreal, PQ / Chesterville, ON  


In [9]:
import math
import numpy as np

#Función para calcular la entropía
def calcular_entropia(datos):
    valores, counts = np.unique(datos, return_counts=True)
    total = len(datos)
    entropia = 0
    for count in counts:
        p = count / total
        entropia -= p * math.log2(p)
    return entropia

#Función para calcular la ganancia de información
def calcular_ganancia_info(datos, atributo):
    #Calcular la entropía original
    entropia_original = calcular_entropia(datos)

    #Obtener los valores únicos del atributo
    valores_atributo = np.unique(atributo)
    ganancia_info = entropia_original

    for valor in valores_atributo:
        #Dividir los datos en función de cada valor del atributo
        datos_subgrupo = datos[atributo == valor]
        proporcion = len(datos_subgrupo) / len(atributo)

        #Restar la entropía del subgrupo
        ganancia_info -= proporcion * calcular_entropia(datos_subgrupo)

    return ganancia_info

#Función para construir el árbol ID3
def construir_arbol_ID3(datos, atributos, clase):
    #Si todos los ejemplos tienen la misma clase, devolver una hoja
    if len(np.unique(datos[clase])) == 1:
        return {'clase': np.unique(datos[clase])[0]}

    #Si no quedan atributos, devolver la clase mayoritaria
    if len(atributos) == 0:
        clase_mayoritaria = datos[clase].mode()[0]
        return {'clase': clase_mayoritaria}

    #Calcular la ganancia de información para cada atributo
    ganancia_maxima = -1
    mejor_atributo = None
    for atributo in atributos:
        ganancia = calcular_ganancia_info(datos[clase], datos[atributo])
        if ganancia > ganancia_maxima:
            ganancia_maxima = ganancia
            mejor_atributo = atributo

    #Crear el nodo del árbol
    arbol = {mejor_atributo: {}}
    valores_atributo = np.unique(datos[mejor_atributo])

    #Llamar recursivamente para cada valor del atributo
    for valor in valores_atributo:
        datos_subgrupo = datos[datos[mejor_atributo] == valor]
        atributos_restantes = [a for a in atributos if a != mejor_atributo]
        arbol[mejor_atributo][valor] = construir_arbol_ID3(datos_subgrupo, atributos_restantes, clase)

    return arbol

#Definir las columnas de atributos y la columna de clase
atributos = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']
clase = 'survived'

#Construir el árbol ID3
arbol = construir_arbol_ID3(data, atributos, clase)

#Mostrar el árbol resultante
print("\nÁrbol de decisión construido por ID3:")
print(arbol)


Árbol de decisión construido por ID3:
{'fare': {0.0: {'age': {19.0: {'clase': 0}, 25.0: {'clase': 1}, 29.8811345124283: {'clase': 0}, 36.0: {'clase': 0}, 38.0: {'clase': 0}, 39.0: {'clase': 0}, 40.0: {'clase': 0}, 49.0: {'pclass': {1: {'clase': 1}, 3: {'clase': 0}}}}}, 3.1708: {'clase': 1}, 4.0125: {'clase': 0}, 5.0: {'clase': 0}, 6.2375: {'clase': 0}, 6.4375: {'clase': 0}, 6.45: {'clase': 0}, 6.4958: {'clase': 0}, 6.75: {'clase': 0}, 6.8583: {'clase': 0}, 6.95: {'sex': {0: {'clase': 0}, 1: {'clase': 1}}}, 6.975: {'age': {27.0: {'clase': 1}, 45.0: {'clase': 0}}}, 7.0: {'sex': {0: {'clase': 0}, 1: {'clase': 1}}}, 7.0458: {'clase': 0}, 7.05: {'age': {20.0: {'clase': 0}, 23.0: {'clase': 0}, 24.0: {'clase': 0}, 25.0: {'clase': 0}, 29.8811345124283: {'pclass': {3: {'sex': {0: {'sibsp': {0: {'parch': {0: {'embarked': {2: {'clase': 0}}}}}}}}}}}, 35.0: {'clase': 0}, 38.0: {'clase': 0}}}, 7.0542: {'clase': 0}, 7.125: {'clase': 0}, 7.1417: {'clase': 1}, 7.225: {'age': {15.0: {'clase': 1}, 20.0:

In [8]:
from sklearn.tree import DecisionTreeClassifier, export_text

#Crear y entrenar el modelo con sklearn
X = data[atributos]
y = data[clase]

#Crear el modelo
modelo = DecisionTreeClassifier(criterion='entropy', random_state=42)

#Entrenar el modelo
modelo.fit(X, y)

# Mostrar el árbol de decisión
print("\nÁrbol de decisión generado con sklearn:")
print(export_text(modelo, feature_names=atributos))

#Evaluación: Predicciones en los datos de entrenamiento
predicciones = modelo.predict(X)

#Evaluar precisión
from sklearn.metrics import accuracy_score
precision = accuracy_score(y, predicciones)
print("\nPrecisión del modelo:", precision)


Árbol de decisión generado con sklearn:
|--- sex <= 0.50
|   |--- age <= 9.50
|   |   |--- sibsp <= 2.50
|   |   |   |--- fare <= 15.57
|   |   |   |   |--- fare <= 13.12
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- fare >  13.12
|   |   |   |   |   |--- fare <= 14.45
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- fare >  14.45
|   |   |   |   |   |   |--- pclass <= 2.50
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- pclass >  2.50
|   |   |   |   |   |   |   |--- age <= 6.50
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- age >  6.50
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |--- fare >  15.57
|   |   |   |   |--- class: 1
|   |   |--- sibsp >  2.50
|   |   |   |--- age <= 3.50
|   |   |   |   |--- age <= 2.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- age >  2.50
|   |   |   |   |   |--- class: 1
|   |   |   |--- age >  3.50
|   |   |   |   |--- class: 0
|   |--- age >  9.50
